In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import json

### What does this do
- ignore all the trajectories in which the relative velocity values are missing
- plot the relative velocity vs time
- plot the velocities (AV and lead vehicule) vs Time
- plot the space gap vs Time

- on the plot velocities (AV and lead vehicule) vs Time, green is for freeflow, yellow for dense traffic, red for the congestion; the red lines indicate the effective change of traffic state

In [10]:

direction = ["east", "west"]

total_iterations = sum(len(os.listdir(f"../data_v2_preprocessed_{d}")) for d in direction)

num_traj = 0
missing = 0
used_num_sec = 0
total_num_sec = 0
sharp_braking = 0
sharp_acc = 0
list_of_dictios = []

with tqdm(total=total_iterations, desc="Processing") as pbar:
    for d in direction:
        base_dir = f"../data_v2_preprocessed_{d}"
        for folder in os.listdir(base_dir):
            folder_path = os.path.join(base_dir, folder)

            if os.path.isdir(folder_path):
                trajectory_file = os.path.join(folder_path, "trajectory.csv")
                
                if os.path.exists(trajectory_file):
                    data = pd.read_csv(trajectory_file)
                    num_traj += 1
                    
                    if 'Time' in data.columns and 'Velocity' in data.columns:

                        total_num_sec += data["Time"].iloc[-1] - data["Time"].iloc[0]
                        
                        # in a lot of cases the RelativeVelocity is missing, so we need to calculate it
                        missing_values = data["RelativeVelocity"].isnull().sum()
                        total_values = len(data["RelativeVelocity"])
                        proportion_missing = (missing_values / total_values) * 100
                        # print(f"Proportion of missing values in RelativeVelocity: {proportion_missing:.2f}%")

                        delta = 10
                        if proportion_missing > 50: 
                            missing += 1
                            pbar.update(1)
                            continue  # remove that line to compute the derivate of the space gap and get the relative velocity
                            # compute the relative velocity by derivating the space gap                     
                            data["RelativeVelocity"] = np.nan  # Initialize with NaN values to handle edges
                            for i in range(delta, len(data["RelativeVelocity"])-delta):
                                data.loc[i, "RelativeVelocity"] = (data.loc[i+delta, "SpaceGap"] - data.loc[i-delta, "SpaceGap"]) / (data.loc[i+delta, "Time"] - data.loc[i-delta, "Time"])
                                inst_acc = (data['Velocity'][i] + data['RelativeVelocity'][i] - data['Velocity'][i-1] - data['RelativeVelocity'][i-1]) / (data['Time'][i] - data['Time'][i-1])
                                inst_acc = inst_acc / 3.6
                                # if inst_acc < -4 or inst_acc > 2:
                                if inst_acc < -20 or inst_acc > 10: # more conservative
                                    data.loc[i, "RelativeVelocity"] = np.nan
                                inst_speed = data['Velocity'][i] + data['RelativeVelocity'][i]
                                if inst_speed < 0 or inst_speed > 140:
                                    data.loc[i, "RelativeVelocity"] = np.nan
                            # data["RelativeVelocity"] = data["RelativeVelocity"].interpolate(method='polynomial', order=3)
                            # data["RelativeVelocity"] = data["RelativeVelocity"].fillna(method='bfill').fillna(method='ffill')

                        used_num_sec += data["Time"].iloc[-1] - data["Time"].iloc[0]

                        fig, ax = plt.subplots(3, 1, figsize=(8, 10))

                        # Plot Time vs Relative Velocity
                        ax[0].plot(data['Time'] - data['Time'][0], data['RelativeVelocity'], label="RelativeVelocity")
                        ax[0].set_xlabel("Time (s)")
                        ax[0].set_ylabel("Velocity (km/h)")
                        ax[0].set_title("Time vs Velocity")
                        ax[0].legend()
                        ax[0].grid(True)

                        # Plot Time vs Velocity (AV and Lead Vehicule)
                        ax[1].plot(data['Time']- data['Time'][0], data['Velocity'] + data['RelativeVelocity'], label="Lead vehicle")
                        ax[1].plot(data['Time']- data['Time'][0], data['Velocity'], label="AV")
                        ax[1].set_xlabel("Time (s)")
                        ax[1].set_ylabel("Velocity (km/h)")
                        ax[1].set_title("Time vs Velocity")
                        ax[1].legend()
                        ax[1].grid(True)

                        ax[1].axhspan(80, ax[1].get_ylim()[1], facecolor='green', alpha=0.3) # Freeflow
                        ax[1].axhspan(40, 80, facecolor='yellow', alpha=0.3) # Dense traffic
                        ax[1].axhspan(0, 40, facecolor='red', alpha=0.3) # Congestion

                        # detection of change of traffic state
                        # theoretical instantaneous maxima: -4 for deceleration and 2 for acceleration
                        data['Acceleration'] = float('nan')
                        delta = 50
                        for i in range(len(data)-delta):
                            acc = (data['Velocity'][i+delta] - data['Velocity'][i]) / (data['Time'][i+delta] - data['Time'][i]) / 3.6 # avg acc on delta/10 sec
                            if acc < -1.5:
                                sharp_braking += 1
                                # ax[1].axvline(x=data['Time'].iloc[i+int(delta/2)] - data['Time'][0], color='red', linestyle='--', linewidth=0.4)
                            elif acc > 1:
                                sharp_acc += 1
                                # ax[1].axvline(x=data['Time'].iloc[i+int(delta/2)] - data['Time'][0], color='green', linestyle='--', linewidth=0.4)
                            data.loc[i, "Acceleration"] = acc

                        # Get the timesetp in between the traffic jam and the freeflow (velocity < 40 km/h or > 80 km/h)
                        # The idea is to ignore the short changes of state (freeflow, dense during 2 sec and then back to freeflow) 
                        dictio_traffic = {"data": folder, "freeflow": [], "congestion": [], "dense": []}
                        current_traffic_state = data['Velocity'].iloc[0]
                        previous_traffic_state = current_traffic_state
                        start = 0
                        start_previous = 0
                        if  current_traffic_state < 40:
                             current_traffic_state = "congestion"
                        elif  current_traffic_state < 80:
                             current_traffic_state = "dense"
                        else:
                             current_traffic_state = "freeflow"

                        line = ax[1].axvline(x=data['Time'].iloc[0] - data['Time'][0], color='black', linestyle='--', linewidth=0.01)
                        for i in range(len(data)):
                            if data['Velocity'].iloc[i] < 40:
                                temp_traffic_state = "congestion"
                            elif data['Velocity'].iloc[i] < 80:
                                temp_traffic_state = "dense"
                            else:
                                temp_traffic_state = "freeflow"
                            if temp_traffic_state != current_traffic_state:
                                if i-start < 100: # only 10s in the same state
                                    if previous_traffic_state == temp_traffic_state:
                                        # we ignore the short state
                                        current_traffic_state = temp_traffic_state
                                        start = start_previous                                        
                                    else:
                                        # it means that we change sharply between congestion and freeflow
                                        current_traffic_state = temp_traffic_state
                                        # we keep the same start as actually it was already in the previous state
                                else:
                                    dictio_traffic[current_traffic_state].append([start,i])
                                    line = ax[1].axvline(x=data['Time'].iloc[start] - data['Time'][0], color='red', linestyle='--', linewidth=0.4)
                                    start_previous, start = start, i
                                    current_traffic_state = temp_traffic_state
                            if i == len(data)-1:
                                dictio_traffic[current_traffic_state].append([start,i])
                                line = ax[1].axvline(x=data['Time'].iloc[start] - data['Time'][0], color='red', linestyle='--', linewidth=0.4)
                                start_previous, start = start, i
                                current_traffic_state = temp_traffic_state

                        
                            
                        # Plot Time vs SpaceGap
                        ax[2].plot(data['Time']- data['Time'][0], data['SpaceGap'])
                        ax[2].set_xlabel("Time (s)")
                        ax[2].set_ylabel("SpaceGap (m)")
                        ax[2].set_title("Time vs SpaceGap")
                        ax[2].grid(True)

                        # Save the plots
                        plt.tight_layout()
                        plot_path = os.path.join(folder_path, "plot.png")
                        plt.savefig(plot_path)
                        plt.close()
                    else:
                        print(f"Required columns not found in {trajectory_file}")
                else:
                    print(f"File trajectory.csv not found in {folder_path}")
            list_of_dictios.append(dictio_traffic)
            pbar.update(1)



Processing: 100%|██████████| 120/120 [01:38<00:00,  1.22it/s]


In [7]:
# save all the timesteps when there is a change of traffic state to a csv
df_traffic_state = pd.DataFrame(list_of_dictios)
df_traffic_state.to_csv('traffic_state.csv', index=False)

In [8]:
# some stats about the traffic to a csv summary.json

time_congestion = 0
time_freeflow = 0
time_dense = 0

for i in range(len(df_traffic_state)):
    for j in range(len(df_traffic_state["congestion"][i])):
        time_congestion += 0.1 * (df_traffic_state["congestion"][i][j][1] - df_traffic_state["congestion"][i][j][0])
    for j in range(len(df_traffic_state["freeflow"][i])):
        time_freeflow += 0.1 * (df_traffic_state["freeflow"][i][j][1] - df_traffic_state["freeflow"][i][j][0])
    for j in range(len(df_traffic_state["dense"][i])):
        time_dense += 0.1 * (df_traffic_state["dense"][i][j][1] - df_traffic_state["dense"][i][j][0])



dictio = {
    "data_origin": ["data_v2_preprocessed_east/", "data_v2_preprocessed_west/"],
    "Total Trajectories": num_traj,
    "Duration (Seconds)": round(total_num_sec, 2),
    "Timesteps/Second": 10,
    "Non-missing relat vel traj": num_traj - missing,
    "Utilized Duration (Seconds)": round(used_num_sec, 2),
    "Time congestion": round(time_congestion, 2),
    "Time freeflow": round(time_freeflow, 2),
    "Time dense": round(time_dense, 2),
    "Percentage congestion": round((time_congestion / used_num_sec) * 100, 2),
    "Percentage freeflow": round((time_freeflow / used_num_sec) * 100, 2),
    "Percentage dense": round((time_dense / used_num_sec) * 100, 2),
    "Number sharp braking": sharp_braking,
    "Number sharp acceleration": sharp_acc
}
# save the dictionary to a json file
with open("summary.json", "w") as json_file:
    json.dump(dictio, json_file, indent=4)